In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import time
import cv2
from os import walk
import triplerecovery as tr
from triplerecovery.constants import LOOKUPS


In [ ]:
imarr = cv2.imread('./images/standard/lena_gray_512.tif', cv2.IMREAD_GRAYSCALE)


In [ ]:
zoomx = 4  # according to the paper the zoom factor is 4
averages = cv2.resize(imarr, None, fx=1/zoomx, fy=1 /
                      zoomx, interpolation=cv2.INTER_AREA)
averages = tr.blocks.make(
    averages, (averages.shape[0]//zoomx, averages.shape[1]//zoomx), addChannel=False)
# averages = averages.reshape(16, 32,32)
averages.shape


In [ ]:
Image.fromarray(averages[9])


In [ ]:
recbits = tr.bits.recovery.make(imarr, LOOKUPS[0], key="")


In [ ]:
ER = tr.embed(imarr)
embedded = ER.imarr.copy()


In [ ]:
Image.fromarray(embedded)


checking the embedding of recovery bits is correct


In [ ]:
exrecbits = tr.bits.recovery.extract(embedded, key="")


In [ ]:
exrecbits.shape, recbits.shape


In [ ]:
(exrecbits == recbits).all()


make image from these bits


In [ ]:
exrecbits = np.packbits(exrecbits, axis=1)
exrecbits.shape


In [ ]:
128*2


In [ ]:
print(embedded.shape)
startx = 0
starty = 0
height = 380
width = 512

for i in range(height):
    for j in range(width):
        embedded[startx+j][starty+i] = 0


In [ ]:
Image.fromarray(embedded)


In [ ]:
# authenticate
AU = tr.authenticate(embedded)


In [ ]:
Image.fromarray(AU.maskarr)


In [ ]:
Image.fromarray(tr.recover(embedded).imarr)

In [ ]:
raise Exception("asd")

In [ ]:
# parasms
lookup = LOOKUPS[0]
recovery_bits = tr.bits.recovery.extract(embedded, key="")
tempred = AU.tempred.copy()


In [ ]:
res = recovery_bits == recbits
for idx, i in enumerate(~res):
    print(idx, np.any(i))


In [ ]:
tempred.shape


In [ ]:
for idx, i in enumerate(tempred):
    print(idx, np.any(i))


In [ ]:
[0, 1, 22]


In [ ]:
# [0, 1, 2, 3],
# [4, 5, 6, 7],
# [8, 9, 10, 11],
# [12, 13, 14, 15]

lookup


In [ ]:
# 0 is tempred
# partner 0 id 0 = 0 partner_block (0, 1) = 7->0,13,10 idx 0 rshape (16, 24576) range 0 8192
# 12 is tempred
# partner 1 id 2 = 12 partner_block (1, 0) = 1->6,12,11 idx 1 rshape (16, 24576) range 8192 16384
# 4 is tempred
# partner 2 id 0 = 4 partner_block (2, 1) = 2->4,9,15 idx 0 rshape (16, 24576) range 0 8192
# 8 is tempred
# partner 3 id 2 = 8 partner_block (3, 0) = 5->3,8,14 idx 1 rshape (16, 24576) range 8192 16384


In [ ]:
imarr = embedded.copy()


In [ ]:
zoomx = 4  # according to the paper the zoom factor is 4
average_bits_imarr = cv2.resize(imarr, None, fx=1/zoomx, fy=1 /
                                zoomx, interpolation=cv2.INTER_AREA)
average_bits_imarr = tr.blocks.make(
    average_bits_imarr, (imarr.shape[0]//zoomx, imarr.shape[1]//zoomx), addChannel=False).reshape(16, -1)
average_bits_imarr = np.unpackbits(average_bits_imarr, axis=1)

# filling from the recovery bits
average_bits = np.zeros(
    (recovery_bits.shape[0], recovery_bits.shape[1]//3), dtype=np.uint8)

for partner in range(lookup.shape[0]):  # A,B,C,D
    for id in range(lookup.shape[1]):  # A1,A2,A3.....D4 etc
        # if not np.any(tempred[lookup[partner, id]]):
        #     # this means that we can pick the averages from the average_bits_imarr
        #     # print("using average bits", partner, id)
        #     # average_bits[lookup[partner, id]
        #     #               ] = average_bits_imarr[lookup[partner, id]]
        #     continue
        print(lookup[partner, id], "is tempred")
        # get the partner block of this id also the id it sits
        partner_block = -1, -1
        for i in range(lookup[partner].shape[0]):
            # if this id isn't tempred, we can stop
            if i != id and not np.any(tempred[lookup[partner, i]]):
                partner_block = partner, i
                break

        if partner_block == (-1, -1):
            print(
                "Could not find partner block for partner", partner, "id", id, "=", lookup[partner, id])
            continue

        # get recovery bits of this id
        # get index of this id in the partner block
        idx = id
        if partner_block[1] <= id:
            idx -= 1

        # length checking
        # if idx*recovery_bits.shape[1]//3 >= recovery_bits.shape[1] or idx*recovery_bits.shape[1]//3+recovery_bits.shape[1]//3 > recovery_bits.shape[1]:
        #     print("Length error for partner", partner,
        #           "id", id, "=", lookup[partner, id])
        #     print("idx", idx, "out of range")
        #     continue

        # now we have partner_block, so we can recover it
        # get the recovery bits of this partner block
        _start = idx*recovery_bits.shape[1]//3
        _end = _start+recovery_bits.shape[1]//3
        print("partner", partner, "id", id, "=", lookup[partner, id], "partner_block", partner_block, "idx",
              idx, "rshape", recovery_bits.shape, "range", _start, _end)
        average_bits[lookup[partner, id]
                     ] = recovery_bits[lookup[partner_block], _start:_end]

# now we have the average bits, so we can recreate the averages and then image

# back to decimals
averages = np.packbits(average_bits, axis=1)
kk = averages.reshape(imarr.shape[0]//4, imarr.shape[1]//4)


In [ ]:
# 0 is tempred
# partner 0 id 0 = 0 partner_block (0, 1) = 7->0,13,10 idx 0 rshape (16, 24576) range 0 8192
# 12 is tempred
# partner 1 id 2 = 12 partner_block (1, 0) = 1->6,12,11 idx 1 rshape (16, 24576) range 8192 16384
# 4 is tempred
# partner 2 id 0 = 4 partner_block (2, 1) = 2->4,9,15 idx 0 rshape (16, 24576) range 0 8192
# 8 is tempred
# partner 3 id 2 = 8 partner_block (3, 0) = 5->3,8,14 idx 1 rshape (16, 24576) range 8192 16384


In [ ]:
averages.shape


In [ ]:
Image.fromarray(tr.blocks.combine(averages.reshape(
    16, 32, 32), (128, 128), (32, 32), channel=False))


In [ ]:
np.packbits(recovery_bits[lookup[0, 1], 0:8192]).reshape(16, -1).shape


In [ ]:
Image.fromarray(tr.blocks.combine(np.packbits(
    recovery_bits[lookup[0, 1], 0:8192]).reshape(16, 8, 8), (32, 32), (8, 8), channel=False))


In [ ]:
Image.fromarray(kk)


In [ ]:
Image.fromarray(cv2.resize((averages[9].reshape(32, 32)),
                           None, fx=4, fy=4, interpolation=cv2.INTER_LANCZOS4))


In [ ]:
x = tr.blocks.combine(averages[9].reshape(
    64, 4, 4), (32, 32), (4, 4), channel=False)
x.shape


In [ ]:
Image.fromarray(x)


In [ ]:
Image.fromarray(cv2.resize(x,
                           None, fx=4, fy=4, interpolation=cv2.INTER_LANCZOS4))
